In [38]:

from google.colab import files
uploaded = files.upload()

In [0]:
import pandas as pd
df = pd.read_csv('./Train.csv')
df.head(2)
#df.apply(lambda x:sum(x.isnull()))
df['Item_Weight'].fillna(df['Item_Weight'].mean(),inplace=True)
df['Outlet_Size'].fillna(df['Outlet_Size'].mode()[0],inplace=True)
#df.head(5)

In [0]:
#df.pivot_table(values='Item_Outlet_Sales',index=['Item_Type','Outlet_Size','Outlet_Type'])

In [41]:
df['Item_Fat_Content'].value_counts()
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})
df['Outlet_Size'].value_counts()

Medium    5203
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [0]:
#categorical variables
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
features = ['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Size','Outlet_Location_Type','Outlet_Type']
label_en = LabelEncoder()

for feature in features:
  df[feature] = label_en.fit_transform(df[feature])
  
df.head(5)
visibility_avg = df.pivot_table(values='Item_Visibility', index='Item_Identifier')
miss_bool = (df['Item_Visibility'] == 0)
df.iloc[[miss_bool],3] = df['Item_Visibility'].mode()[0]
df['Outlet_Establishment_Year'] = 2013 - df['Outlet_Establishment_Year']
#df['Outlet_Establishment_Year'].describe()


In [0]:
#one hot encoding
df = pd.get_dummies(df, columns=['Item_Fat_Content','Item_Type','Outlet_Type','Outlet_Location_Type','Outlet_Size'],drop_first = True)

y = df.iloc[:,6]
df.drop(['Item_Outlet_Sales','Outlet_Identifier','Item_Identifier'],axis=1,inplace=True)
x = df.iloc[:,:]


In [44]:
#backward elimination
import statsmodels.formula.api as sm
import numpy as np
def backwardElimination(x,sl):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_ols = sm.OLS(y, x).fit()
        maxVar = max(regressor_ols.pvalues)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_ols.pvalues[j] == maxVar):
                    x = np.delete(x,j,1)
    #print(regressor_ols.summary())
    return x

x = np.append(arr=np.ones((len(x),1)).astype(int),values=x,axis=1)
sl = 0.01
x_opt = x
x_opt = backwardElimination(x_opt, sl)
x_opt.shape

(8523, 9)

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,accuracy_score
from math import sqrt
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_opt,y,test_size = 0.2,random_state=0)
#regression models
#linear regression

lr = LinearRegression(normalize=True)
lr.fit(x_train,y_train)
pred = lr.predict(x_test)


In [46]:
x_train_copy, y_train_copy = x_train, y_train

#feature scaling
from sklearn.preprocessing import StandardScaler
s_x = StandardScaler()
s_y = StandardScaler()
x_train_copy = s_x.fit_transform(x_train_copy)
y_train_copy = s_y.fit_transform(y_train_copy.values.reshape(-1,1))

#svr regression
from sklearn.svm import SVR
svr = SVR()
svr.fit(x_train_copy,y_train_copy)
pred = s_y.inverse_transform(svr.predict(s_x.fit_transform(x_test)))
rmse = sqrt(mean_squared_error(y_test, pred))
print("svr:",rmse)


('svr:', 1081.6794356077455)


In [47]:
#decision tree 
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=15, min_samples_leaf=100,random_state = 2)
dt.fit(x_train,y_train)
pred = dt.predict(x_test) 
rmse = sqrt(mean_squared_error(y_test, pred))
print("Decison tree:",rmse)

('Decison tree:', 1092.6739810211964)


In [48]:
#xgboost
from xgboost.sklearn import XGBRegressor
XGB = XGBRegressor()
XGB.fit(x_train, y_train)
pred = XGB.predict(x_test)
pred = pred.astype(int) 
rmse = sqrt(mean_squared_error(y_test, pred))
print("XGBoost:",rmse)

('XGBoost:', 1102.2411335659456)
